In [56]:
%matplotlib inline
import datetime
import time
import numpy as np
import pandas as pd
import pickle
from sklearn import preprocessing

In [57]:
def adjust_id(df_element):
    adj = df_element.replace('email','').replace('ip','').replace('card','')
    return int(float(adj))

def string_to_timestamp(df_row):
    time_stamp = time.strptime(df_row.creationdate, '%Y-%m-%d %H:%M:%S')
    return time.mktime(time_stamp)

def adjust_creationdate(df_row):
    year_info = datetime.datetime.strptime(df_row.creationdate,'%Y-%m-%d %H:%M:%S').year
    month_info = datetime.datetime.strptime(df_row.creationdate,'%Y-%m-%d %H:%M:%S').month
    day_info = datetime.datetime.strptime(df_row.creationdate,'%Y-%m-%d %H:%M:%S').day
    creationdate = str(year_info)+'-'+str(month_info)+'-'+str(day_info)#Date of transaction 
    return creationdate

def mean_distance_amount(df_row):
    return df_row.amount_mean - df_row.amount

In [58]:
src = 'data_for_student_case.csv'
df = pd.read_csv(src)


df = df[(df['simple_journal'] != 'Refused') & ~(df['mail_id'].str.contains("na", case=False)) & ~(df['bin'].isnull())]
adjust_id_cols = ['mail_id', 'ip_id', 'card_id']
df[adjust_id_cols] = df[adjust_id_cols].applymap(adjust_id)

df['cvcresponsecode'] = df.apply(lambda x: 3 if x.cvcresponsecode > 2 else x.cvcresponsecode, axis = 1)
df['label'] = df.apply(lambda x: 1 if x.simple_journal == "Chargeback" else 0, axis = 1)
df['creationdate_timestamp'] = df.apply(string_to_timestamp, axis = 1)
df['creationdate'] = df.apply(adjust_creationdate, axis = 1)
df['creationdate'] = pd.to_datetime(df['creationdate'])
df.loc[:,('issuercountrycode')] = df['issuercountrycode'].fillna('QQ') # We add a custom value for unknown codes

df_temp = df[['mail_id','amount']].groupby('mail_id').agg(['max','mean'])
df_mean = df_temp['amount']['mean']
df = df.join(df_mean, on='mail_id')
df = df.rename(columns={'mean': 'amount_mean'})
df['amount_distance'] = df.apply(mean_distance_amount, axis=1)

print(df.dtypes)
df.tail()

txid                                     int64
bookingdate                             object
issuercountrycode                       object
txvariantcode                           object
bin                                    float64
amount                                 float64
currencycode                            object
shoppercountrycode                      object
shopperinteraction                      object
simple_journal                          object
cardverificationcodesupplied            object
cvcresponsecode                          int64
creationdate                    datetime64[ns]
accountcode                             object
mail_id                                  int64
ip_id                                    int64
card_id                                  int64
label                                    int64
creationdate_timestamp                 float64
amount_mean                            float64
amount_distance                        float64
dtype: object

,txid,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,simple_journal,...,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationdate_timestamp,amount_mean,amount_distance
290377,482729,2015-11-03 18:57:01,SE,visadebit,453903.0,48100.0,SEK,SE,Ecommerce,Settled,...,1,2015-10-31,SwedenAccount,135821,125573,83162,0,1.446330e+09,48100.000000,0.000000
290378,482731,2015-11-03 08:12:14,SE,mcdebit,554501.0,195000.0,SEK,SE,Ecommerce,Settled,...,1,2015-10-31,SwedenAccount,259702,202029,131519,0,1.446331e+09,195000.000000,0.000000
290379,482732,2015-11-03 08:12:14,SE,visadebit,453903.0,52500.0,SEK,SE,Ecommerce,Settled,...,1,2015-10-31,SwedenAccount,32927,229869,142173,0,1.446331e+09,52500.000000,0.000000
290380,482733,2015-11-03 18:56:44,SE,mcdebit,554501.0,87700.0,SEK,SE,Ecommerce,Settled,...,1,2015-10-31,SwedenAccount,97400,200721,141579,0,1.446332e+09,87700.000000,0.000000
290381,482734,2015-11-03 08:12:14,SE,mcdebit,554501.0,58500.0,SEK,SE,Ecommerce,Settled,...,1,2015-10-31,SwedenAccount,12831,199739,269508,0,1.446332e+09,60716.666667,2216.666667


In [59]:
df_temp = df[['mail_id', 'card_id']].groupby('mail_id').count()
df_temp = df_temp.rename(columns={'card_id': 'total_cards_used'})
df = df.join(df_temp, on='mail_id')
df.head()

,txid,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,simple_journal,...,creationdate,accountcode,mail_id,ip_id,card_id,label,creationdate_timestamp,amount_mean,amount_distance,total_cards_used
0,1,2015-11-09 14:26:51,MX,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-01,MexicoAccount,68370,111778,184798,1,1.435785e+09,64800.0,0.0,1
1,2,2015-11-09 14:27:38,MX,mccredit,547046.0,44900.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-02,MexicoAccount,101299,78749,151595,1,1.435805e+09,44900.0,0.0,1
2,3,2015-11-23 16:34:16,MX,mccredit,528843.0,149900.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-02,MexicoAccount,278604,70594,242142,1,1.435840e+09,149900.0,0.0,1
3,4,2015-11-23 16:34:51,MX,mccredit,547146.0,109900.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-03,MexicoAccount,47409,113648,181744,1,1.435903e+09,109900.0,0.0,1
4,5,2015-11-09 14:26:08,MX,visaclassic,477291.0,89900.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-08,MexicoAccount,205501,83553,97271,1,1.436373e+09,89900.0,0.0,1


In [60]:
le = preprocessing.LabelEncoder()
le.fit(df['issuercountrycode'])
df.loc[:,('issuercountrycode')] = le.transform(df['issuercountrycode'])
df.head()

,txid,bookingdate,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,simple_journal,...,creationdate,accountcode,mail_id,ip_id,card_id,label,creationdate_timestamp,amount_mean,amount_distance,total_cards_used
0,1,2015-11-09 14:26:51,76,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-01,MexicoAccount,68370,111778,184798,1,1.435785e+09,64800.0,0.0,1
1,2,2015-11-09 14:27:38,76,mccredit,547046.0,44900.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-02,MexicoAccount,101299,78749,151595,1,1.435805e+09,44900.0,0.0,1
2,3,2015-11-23 16:34:16,76,mccredit,528843.0,149900.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-02,MexicoAccount,278604,70594,242142,1,1.435840e+09,149900.0,0.0,1
3,4,2015-11-23 16:34:51,76,mccredit,547146.0,109900.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-03,MexicoAccount,47409,113648,181744,1,1.435903e+09,109900.0,0.0,1
4,5,2015-11-09 14:26:08,76,visaclassic,477291.0,89900.0,MXN,MX,Ecommerce,Chargeback,...,2015-07-08,MexicoAccount,205501,83553,97271,1,1.436373e+09,89900.0,0.0,1


In [61]:
print(le.classes_)
with open('issuercountry_le.pickle', 'wb') as f:
    pickle.dump(le, f)

['AE' 'AI' 'AL' 'AM' 'AO' 'AR' 'AT' 'AU' 'AZ' 'BA' 'BB' 'BD' 'BE' 'BG' 'BH'
 'BM' 'BN' 'BR' 'BW' 'BY' 'BZ' 'CA' 'CH' 'CK' 'CL' 'CN' 'CO' 'CR' 'CW' 'CY'
 'DE' 'DK' 'DO' 'EC' 'EE' 'EG' 'ES' 'FI' 'FJ' 'FR' 'GB' 'GE' 'GH' 'GI' 'HK'
 'HN' 'HR' 'HU' 'ID' 'IE' 'IL' 'IN' 'IQ' 'IS' 'IT' 'JO' 'JP' 'KE' 'KH' 'KR'
 'KW' 'KZ' 'LA' 'LB' 'LK' 'LT' 'LU' 'LV' 'LY' 'MD' 'ME' 'MK' 'MN' 'MO' 'MT'
 'MU' 'MX' 'MY' 'NG' 'NL' 'NO' 'NZ' 'OM' 'PA' 'PE' 'PH' 'PK' 'PL' 'PT' 'PY'
 'QA' 'QQ' 'RO' 'RS' 'RU' 'SA' 'SB' 'SE' 'SG' 'SI' 'SK' 'SL' 'TH' 'TJ' 'TM'
 'TR' 'TW' 'TZ' 'UA' 'US' 'UY' 'VG' 'VN' 'ZA' 'ZM' 'ZW' 'ZZ']


In [62]:
df_less = df.drop(['txid', 'bookingdate', 'simple_journal', 'amount_mean'], axis=1)
#df_less = df_less.sort_values(by=['creationdate'], ascending=True) Not really needed
df_less.head()

,issuercountrycode,txvariantcode,bin,amount,currencycode,shoppercountrycode,shopperinteraction,cardverificationcodesupplied,cvcresponsecode,creationdate,accountcode,mail_id,ip_id,card_id,label,creationdate_timestamp,amount_distance,total_cards_used
0,76,mccredit,530056.0,64800.0,MXN,MX,Ecommerce,True,0,2015-07-01,MexicoAccount,68370,111778,184798,1,1.435785e+09,0.0,1
1,76,mccredit,547046.0,44900.0,MXN,MX,Ecommerce,True,0,2015-07-02,MexicoAccount,101299,78749,151595,1,1.435805e+09,0.0,1
2,76,mccredit,528843.0,149900.0,MXN,MX,Ecommerce,True,0,2015-07-02,MexicoAccount,278604,70594,242142,1,1.435840e+09,0.0,1
3,76,mccredit,547146.0,109900.0,MXN,MX,Ecommerce,True,0,2015-07-03,MexicoAccount,47409,113648,181744,1,1.435903e+09,0.0,1
4,76,visaclassic,477291.0,89900.0,MXN,MX,Ecommerce,True,0,2015-07-08,MexicoAccount,205501,83553,97271,1,1.436373e+09,0.0,1


In [63]:
with open('dataframe.pickle', 'wb') as f:
    pickle.dump(df_less, f)

In [64]:
(issuercountry_set, txvariantcode_set, currencycode_set, shoppercountry_set, interaction_set,
verification_set, accountcode_set, mail_id_set, ip_id_set, card_id_set) = [set() for _ in xrange(10)]
(issuercountry_dict, txvariantcode_dict, currencycode_dict, shoppercountry_dict, interaction_dict,
verification_dict, accountcode_dict, mail_id_dict, ip_id_dict, card_id_dict) = [{} for _ in xrange(10)]
issuercountry_set = df_less.issuercountrycode.unique()
txvariantcode_set = df_less.txvariantcode.unique()
currencycode_set = df_less.currencycode.unique()
shoppercountry_set = df_less.shoppercountrycode.unique()
interaction_set = df_less.shopperinteraction.unique()
verification_set = df_less.cardverificationcodesupplied.unique()
accountcode_set = df_less.accountcode.unique()
mail_id_set = df_less.mail_id.unique()
ip_id_set = df_less.ip_id.unique()
card_id_set = df_less.card_id.unique()

In [65]:
with open('feature_sets.pickle', 'wb') as f:
    pickle.dump((issuercountry_set,txvariantcode_set,currencycode_set,shoppercountry_set,interaction_set,verification_set,accountcode_set,mail_id_set,ip_id_set,card_id_set), f)